In [1]:
import base64
import datetime
import itertools
import json
import math
import operator
import os
import pickle
import random
import re
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
d = pd.concat([pd.read_csv('tmp/train_ids.csv', index_col='id'),
               pd.read_csv('tmp/test_ids.csv', index_col='id')])
print d.dtypes

interest_level    object
dtype: object


In [3]:
e = pd.read_csv('tmp/features_bathrooms.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

bathrooms    float64
dtype: object


In [4]:
e = pd.read_csv('tmp/features_bedrooms.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

bedrooms    float64
dtype: object


In [5]:
e = pd.read_csv('tmp/features_building_id.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

building_id_is_zero     float64
num_apts_in_building    float64
building_code             int64
dtype: object


In [6]:
e = pd.read_csv('tmp/features_created.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

created_month                   int64
created_day_of_month            int64
created_dow                     int64
created_hour                  float64
created_minute                float64
created_minute_of_the_day     float64
created_hour_of_dataset       float64
created_distance_from_peak    float64
num_apts_same_3hours          float64
num_apts_prev_3hours          float64
num_apts_next_3hours          float64
num_apts_same_2days           float64
num_apts_prev_2days           float64
num_apts_next_2days           float64
dtype: object


In [7]:
e = pd.read_csv('tmp/features_description.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

desc_count_non_alphanumeric_characters    float64
desc_num_bangs                            float64
desc_num_breaks                           float64
desc_num_emails                           float64
desc_num_nonalpha                         float64
desc_num_paragraphs                       float64
desc_num_parentheses                      float64
desc_num_phones                           float64
desc_num_website_redacted                 float64
desc_num_words                            float64
num_characters                            float64
ratio_non_alphanumeric_characters         float64
desc_num_words_q                          float64
desc_count_24                               int64
desc_count_apartment                        int64
desc_count_appliances                       int64
desc_count_area                             int64
desc_count_bathroom                         int64
desc_count_beautiful                        int64
desc_count_bedroom                          int64


In [8]:
e = pd.read_csv('tmp/features_display_address.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

daddr_num_words             float64
daddr_has_number              int64
daddr_has_ordinal             int64
daddr_num_upper_words       float64
daddr_count_10th              int64
daddr_count_2nd               int64
daddr_count_34th              int64
daddr_count_37th              int64
daddr_count_ave               int64
daddr_count_avenue            int64
daddr_count_broadway          int64
daddr_count_east              int64
daddr_count_end               int64
daddr_count_gold              int64
daddr_count_john              int64
daddr_count_park              int64
daddr_count_place             int64
daddr_count_second            int64
daddr_count_st                int64
daddr_count_street            int64
daddr_count_wall              int64
daddr_count_washington        int64
daddr_count_west              int64
daddr_count_york              int64
num_apts_with_same_daddr    float64
dtype: object


In [9]:
e = pd.read_csv('tmp/features_features.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

num_features                                 float64
top_10_feature_coverage                      float64
top_10_feature_ratio                         float64
features_count_balcony                         int64
features_count_cats_allowed                    int64
features_count_common_outdoor_space            int64
features_count_dining_room                     int64
features_count_dishwasher                      int64
features_count_dogs_allowed                    int64
features_count_doorman                         int64
features_count_dryer_in_unit                   int64
features_count_elevator                        int64
features_count_exclusive                       int64
features_count_fireplace                       int64
features_count_fitness_center                  int64
features_count_furnished                       int64
features_count_garage                          int64
features_count_garden                          int64
features_count_green_building                 

In [10]:
e = pd.read_csv('tmp/features_latlon.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

latitude             float64
longitude            float64
cluster_id             int64
cluster_size         float64
distance_cc          float64
price_clust_br_q     float64
area_density         float64
area_distance_com    float64
dtype: object


In [11]:
e = pd.read_csv('tmp/features_manager_id.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

num_apts_at_manager      float64
num_apts_at_manager_q    float64
private_manager            int64
manager_code               int64
dtype: object


In [12]:
e = pd.read_csv('tmp/features_photos.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

num_photos         float64
has_photos           int64
photos_per_room    float64
dtype: object


In [13]:
e = pd.read_csv('tmp/features_price.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

price                              float64
price_br_q                         float64
num_apts_with_approx_same_price    float64
dtype: object


In [14]:
e = pd.read_csv('tmp/features_street_address.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

saddr_num_words             float64
saddr_has_number              int64
saddr_has_ordinal             int64
saddr_num_upper_words       float64
daddr_saddr_same              int64
num_apts_with_same_saddr    float64
saddr_count_10                int64
saddr_count_100               int64
saddr_count_150               int64
saddr_count_200               int64
saddr_count_25                int64
saddr_count_34th              int64
saddr_count_37th              int64
saddr_count_50                int64
saddr_count_90                int64
saddr_count_95                int64
saddr_count_ave               int64
saddr_count_avenue            int64
saddr_count_broadway          int64
saddr_count_east              int64
saddr_count_place             int64
saddr_count_st                int64
saddr_count_street            int64
saddr_count_wall              int64
saddr_count_washington        int64
saddr_count_west              int64
dtype: object


In [15]:
e = pd.read_csv('tmp/features_others.csv', index_col='id')
print e.dtypes
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

price_per_bedrooms                   float64
rooms_diff                           float64
rooms_sum                            float64
bathrooms_per_bedrooms               float64
hours_since_managers_last_listing    float64
dtype: object


In [16]:
d.interest_level.value_counts()

low       34284
medium    11229
high       3839
Name: interest_level, dtype: int64

In [17]:
for c in d.columns:
    if c != 'interest_level' and (d.dtypes[c] not in ('int64', 'float64')):
        del d[c]

In [18]:
tr = d[d.interest_level.notnull()]
print len(tr)

49352


In [19]:
te = d[d.interest_level.isnull()]
print len(te)

74659


In [20]:
print tr.values.shape
print te.values.shape

(49352, 212)
(74659, 212)


In [21]:
# target_num_map = {'high':0, 'medium':1, 'low':2}
# tr = np.array(trl.apply(lambda x: target_num_map[x]))

In [22]:
tr.to_csv('tmp/combined_train.csv')
te.to_csv('tmp/combined_test.csv')